In [1]:
import os,time
os.environ['CUDA_VISIBLE_DEVICES']='0'
import copy
import logging
import numpy as np
import pandas as pd
from datetime import datetime,timedelta,date
import time
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, AutoModel, get_linear_schedule_with_warmup, \
    default_data_collator, TrainingArguments, Trainer
from sentence_transformers import SentencesDataset, LoggingHandler, SentenceTransformer
from sklearn.metrics.pairwise import paired_cosine_distances, paired_euclidean_distances, paired_manhattan_distances
from scipy.stats import pearsonr, spearmanr
from transformers import AutoTokenizer
from transformers import BertConfig, BertTokenizer,BertTokenizerFast
from datasets import Dataset
from typing import List,Union
import tqdm
import re
import json
import faiss
os.chdir('/home/stops/Work_space/NLP_work/Med_assit_chatglm')

from db_config_taiyi import DB ## load data from pgsql

logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO,format='%(asctime)s - %(levelname)s - %(message)s')
logger.info('Starting')

"""
1.针对症状和疾病，按广义症状进行相似度判断。优化训练集和相似模型。 -231120
腹痛>上腹痛>>大腿疼

2.症状集合："output_data/Symptom_SET_keep_df_1110.xlsx" 8k

3.症状相似度训练集：Med_general_norm_category_1p7w_220929.xlsx
       迭代数据集：Med_general_norm_category_231121.xlsx

4.


5.


"""


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/stops/miniconda3/envs/LLM_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/stops/miniconda3/envs/LLM_env/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


2023-11-22 16:35:36,344 - INFO : Starting


'\n1.针对症状和疾病，按广义症状进行相似度判断。优化训练集和相似模型。 -231120\n腹痛>上腹痛>>大腿疼\n\n2.症状集合："output_data/Symptom_SET_keep_df_1110.xlsx" 8k\n\n3.症状相似度训练集：Med_general_norm_category_1p7w_220929.xlsx\n\n4.\n\n5.\n\n\n'

In [2]:
def show_df(df):
    print(df.shape)
    print(df.head(2))
    print(df.isnull().sum())

In [3]:
## 真实问诊初步去重的症状
symptom_set_file="output_data/Symptom_SET_keep_df_1110.xlsx"
symptom_set_df=pd.read_excel(symptom_set_file)
show_df(symptom_set_df)
logger.info(f"load data file: {symptom_set_file}")

2023-11-22 16:35:36,569 - INFO : load data file: output_data/Symptom_SET_keep_df_1110.xlsx


(7979, 1)
  symptom
0   小便后刺痛
1   被仓鼠咬伤
symptom    0
dtype: int64


In [4]:
## 训练集汇总出现的症状
symptom_train_file="data/simi_data/Med_general_norm_category_1p7w_220929.xlsx"
symptom_train_df=pd.read_excel(symptom_train_file)

show_df(symptom_train_df)
logger.info(f"load data file: {symptom_train_file}")


2023-11-22 16:35:37,290 - INFO : load data file: data/simi_data/Med_general_norm_category_1p7w_220929.xlsx


(17706, 4)
  Category Stand_text Simi_text  label
0   疾病-皮肤科     EB病毒感染    感染eb病毒      1
1   疾病-皮肤科     EB病毒感染   感染过eb病毒      1
Category      0
Stand_text    0
Simi_text     0
label         0
dtype: int64


In [5]:
"""
目标：优化训练集和相似模型-231122
    突出广义症状：腹痛和下腹痛相同。不同于重症中细分症状。

1.拿真实问诊症状去匹配自身剩余症状，如有问题，再去训练语料找到标准症状找到对应语料，审核正负样本数据集。


"""

train_symptom_set_list=symptom_train_df["Stand_text"].unique().tolist()

logger.info(f"train_symptom_set_list nums: {len(train_symptom_set_list)}")
logger.info(f"examples: {train_symptom_set_list[:20]}")


2023-11-22 16:35:37,295 - INFO : train_symptom_set_list nums: 446
2023-11-22 16:35:37,295 - INFO : examples: ['EB病毒感染', 'HPV病毒感染', '癌症', '白痰', '白细胞异常', '包茎', '包块', '包皮过长', '包皮系带过短', '鼻出血', '鼻窦炎', '鼻干', '鼻塞', '鼻息气热', '鼻息肉', '鼻炎', '鼻咽部疾病', '鼻痒', '鼻音重', '鼻中隔偏曲']


In [6]:
## 通用症状
symptom_general_file="data/simi_data/Symptom_general_norm_1p1w_220923.xlsx"
symptom_general_df=pd.read_excel(symptom_general_file,index_col=0)

show_df(symptom_general_df)
logger.info(f"load data file: {symptom_general_file}")
logger.info(f"set nums: {symptom_general_df['Stand_text'].unique().size}")


2023-11-22 16:35:37,708 - INFO : load data file: data/simi_data/Symptom_general_norm_1p1w_220923.xlsx
2023-11-22 16:35:37,709 - INFO : set nums: 278


(11332, 3)
  Stand_text Simi_text  label
1         畏寒      有点畏寒      1
2         畏寒      畏寒怕冷      1
Stand_text    0
Simi_text     0
label         0
dtype: int64


In [7]:
## 通用疾病
disease_general_file="data/simi_data/Disease_general_norm_6k_220929.xlsx"
disease_general_df=pd.read_excel(disease_general_file,index_col=0)

show_df(disease_general_df)
logger.info(f"load data file: {disease_general_file}")
logger.info(f"set nums: {disease_general_df['Stand_text'].unique().size}")


2023-11-22 16:35:38,095 - INFO : load data file: data/simi_data/Disease_general_norm_6k_220929.xlsx
2023-11-22 16:35:38,095 - INFO : set nums: 173


(6298, 3)
  Stand_text  Simi_text  label
1  营养性维生素D缺乏  有一点维生素d缺乏      1
2  营养性维生素D缺乏    维生素d有缺乏      1
Stand_text    0
Simi_text     0
label         0
dtype: int64


In [8]:
#############################
## load simi model
#############################
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu' )
#simi_model_path='/home/stops/Work_space/NLP_models/Medical_general_simi_model'
#simi_model_path='/home/stops/Work_space/NLP_work/Med_Doctor_diagnosis_query/Disease_10_Doctor_query_v2_1009/expl_simi_model/Medical_simi-inner_query--epochs_3-2022-10-14_18-05-29'
#simi_model_path="/home/stops/Work_space/NLP_models/Erlangshen-Roberta-330M-Similarity"
#simi_model_path="/home/stops/Work_space/NLP_models/bge-large-zh-noinstruct"
#simi_model_path="/home/stops/Work_space/NLP_models/bge-base-zh-v1.5"

#simi_model_path="simi_model/Medical_simi-_all_-epochs_3-2023-11-13_18-45-25"
simi_model_path="simi_model/Medical_simi-_General_-epochs_3-2023-11-22" ## update-version

simi_model=SentenceTransformer(simi_model_path,device=device)
simi_model.eval()

logger.info(f'load simi model: {simi_model_path}')


2023-11-22 16:35:38,100 - INFO : Load pretrained SentenceTransformer: simi_model/Medical_simi-_General_-epochs_3-2023-11-22
2023-11-22 16:35:38,915 - INFO : load simi model: simi_model/Medical_simi-_General_-epochs_3-2023-11-22


In [9]:
import faiss

def load_faiss_search_engine(object_embed:torch.Tensor):
    s_time = time.time()
    d = object_embed.shape[1]
    ## CPU mode
    index = faiss.index_factory(d, 'Flat', faiss.METRIC_INNER_PRODUCT)  # build the index
    faiss.normalize_L2(object_embed)
    index.add(object_embed)
    logger.info('Loading CPU faiss cost time : {:.4f} (seconds)'.format(time.time() - s_time))
    return index

In [10]:
def get_simi_embed(conversation_flatten_text_list):
    run_all_nums=len(conversation_flatten_text_list)
    logger.info(f'simi model process data: {run_all_nums}')
    s_time=time.time()
    corpus_matrix  = []
    batch_size=1000
    run_all_steps=len(list(range(0,len(conversation_flatten_text_list), batch_size)))
    logger.info(f'batch_size   : {batch_size}')
    logger.info(f'run_all_steps: {run_all_steps}')

    for i in range(0,run_all_nums, batch_size):
        if i%1000==0:
            logger.info('run step {}, finished: {:.2%}'.format(i,i/run_all_nums))
        corpus_matrix.append(simi_model.encode(conversation_flatten_text_list[i:i+batch_size],batch_size=batch_size,show_progress_bar=False))
    corpus_embed =np.concatenate(corpus_matrix)
    logger.info('build embed cost time: {:.2f}s, shape: {}'.format(time.time()-s_time,corpus_embed.shape))
    return corpus_embed

In [11]:
####################
## build-Embedding
####################

symptom_set_flatten_list=symptom_set_df["symptom"].unique().tolist()
symptom_set_embed=get_simi_embed(symptom_set_flatten_list)

2023-11-22 16:35:38,931 - INFO : simi model process data: 7655
2023-11-22 16:35:38,931 - INFO : batch_size   : 1000
2023-11-22 16:35:38,932 - INFO : run_all_steps: 8
2023-11-22 16:35:38,932 - INFO : run step 0, finished: 0.00%
2023-11-22 16:35:41,253 - INFO : run step 1000, finished: 13.06%
2023-11-22 16:35:41,525 - INFO : run step 2000, finished: 26.13%
2023-11-22 16:35:41,787 - INFO : run step 3000, finished: 39.19%
2023-11-22 16:35:42,158 - INFO : run step 4000, finished: 52.25%
2023-11-22 16:35:42,445 - INFO : run step 5000, finished: 65.32%
2023-11-22 16:35:42,799 - INFO : run step 6000, finished: 78.38%
2023-11-22 16:35:43,133 - INFO : run step 7000, finished: 91.44%
2023-11-22 16:35:43,357 - INFO : build embed cost time: 4.43s, shape: (7655, 768)


In [12]:
symptom_set_faiss_index=load_faiss_search_engine(symptom_set_embed)

2023-11-22 16:35:43,370 - INFO : Loading CPU faiss cost time : 0.0090 (seconds)


In [13]:
##########################################
##  Retrieve  Test
##########################################
## Step-1: 首先针对真实问诊中症状，匹配自身剩余其他症状，查看其最相似值

top_k=50

test_text="鼻痒"
simi_threshold=0.86
test_embed=simi_model.encode([test_text])
faiss.normalize_L2(test_embed)
test_Proba, test_Id = symptom_set_faiss_index.search(test_embed, top_k)
logger.info(f"test_text : {test_text}")
logger.info(f"test_Proba: {test_Proba}")
logger.info(f"test_Id   : {test_Id}")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-11-22 16:35:43,407 - INFO : test_text : 鼻痒
2023-11-22 16:35:43,409 - INFO : test_Proba: [[1.         0.9945984  0.842765   0.7762667  0.76982427 0.762655
  0.7606481  0.7550805  0.7302172  0.72369456 0.7199323  0.7162697
  0.69906163 0.6905434  0.67803925 0.668731   0.6631844  0.6624191
  0.66239005 0.66184473 0.65839934 0.6548296  0.6444884  0.6262642
  0.6194328  0.6017078  0.60128105 0.5923245  0.58138335 0.58131814
  0.58069015 0.57331574 0.5649778  0.5544405  0.55365705 0.5498495
  0.5475721  0.5436986  0.52993226 0.5296795  0.52590597 0.5233761
  0.5226804  0.5217592  0.52055854 0.5186656  0.5171307  0.5109539
  0.51084286 0.51045203]]
2023-11-22 16:35:43,410 - INFO : test_Id   : [[6611 1370 2116 6845 6494 7021 5456 4900 6122 2036 5502 4534 3415 4271
  7648 1013 4188 3581 1976   55 7520 6093 4752 3071 4615 6088 2364 4867
  7063 6297 7548 6710 3537 1509 4292  613 7020  488 7048 2600 1587 1123
  7032 6550 2091 3698 6192 2204 1847 5952]]


In [14]:
for sub_proba_list,sub_idx_list in zip(test_Proba,test_Id):
    for sub_proba, sub_text_idx in zip(sub_proba_list,sub_idx_list):
        if sub_proba>=simi_threshold:
            print("Simi_Proba: {:.4f} , Simi_Text: {}".format(sub_proba,symptom_set_flatten_list[sub_text_idx]))


Simi_Proba: 1.0000 , Simi_Text: 鼻痒
Simi_Proba: 0.9946 , Simi_Text: 鼻子痒


In [15]:
## Step-2: 针对上述异常值，检索训练集中数据，针对性正价正负样本。


class FocusInfo():
    def __init__(self,focus_text_list,simi_model=simi_model):
        logger.info('Initializing focus_text_list ')
        self.stand_text=list(set(focus_text_list))
        self.stand_embed=simi_model.encode(self.stand_text,batch_size=300,show_progress_bar=False)
        logger.info('embed shape: {}'.format(self.stand_embed.shape))
        self.stand_nums=len(self.stand_text)
        self.text2id=dict(zip(self.stand_text,list(range(self.stand_nums))))
        self.id2text=dict(zip(self.text2id.values(),self.text2id.keys()))

## 训练集标准症状集合
standSymptomInfo=FocusInfo(train_symptom_set_list)


2023-11-22 16:35:43,420 - INFO : Initializing focus_text_list 
2023-11-22 16:35:43,486 - INFO : embed shape: (446, 768)


In [16]:
from text_similarity_calculate import simi_batch_match_info_infer

text_batch_list=["咳嗽","鼻痒","发烧"]
simi_threshold=0.86
top_k=5
test_res=simi_batch_match_info_infer(text_batch_list,standSymptomInfo,simi_model,thre=simi_threshold,top_k=top_k,batch_size=100)
print("test_res: ")
print(test_res)

for sub_key in test_res:
    sub_symptom=text_batch_list[sub_key]
    sub_res_list=test_res[sub_key]
    print("symptom: ",sub_symptom)
    for sub_res_tuple in sub_res_list:
        print("    matched symptom: ",standSymptomInfo.stand_text[sub_res_tuple[1]])
        print("    matched probability: ",sub_res_tuple[0])


test_res: 
{1: [(1.0, 264)], 2: [(0.96, 88)]}
symptom:  鼻痒
    matched symptom:  鼻痒
    matched probability:  1.0
symptom:  发烧
    matched symptom:  发热
    matched probability:  0.96


In [17]:
"""
被仓鼠咬伤，指甲发黑

剔除宽泛症状：-231122
流血，

"""


'\n被仓鼠咬伤，指甲发黑\n\n剔除宽泛症状：-231122\n流血，\n\n'

In [18]:
##########################################
## Stage-1: Retrieve  Symptom-SET
##########################################

simi_threshold=0.80
top_k=120

scan_window_size=5
start_idx=190

scan_symptom_set_list=symptom_set_flatten_list[start_idx:start_idx+scan_window_size]
logger.info(f"scan_symptom_set_list : {scan_symptom_set_list}")
test_embed=simi_model.encode(scan_symptom_set_list)
faiss.normalize_L2(test_embed)
test_Proba, test_Id = symptom_set_faiss_index.search(test_embed, top_k)
logger.info(f"test_bums : {len(test_Proba)}")
sub_i=0
for sub_proba_list,sub_idx_list in zip(test_Proba,test_Id):
    print("*****"*10)
    print(f"current symptom: {sub_i}  {scan_symptom_set_list[sub_i]}")
    for sub_proba, sub_text_idx in zip(sub_proba_list,sub_idx_list):
        if sub_proba>=simi_threshold:
            print("Simi_Proba: {:.4f} , Simi_Text: {}".format(sub_proba,symptom_set_flatten_list[sub_text_idx]))
    sub_i+=1
    start_idx+=1

print("*****"*4)
print(f"Next start index: {start_idx}")
print("*****"*4)


2023-11-22 16:35:43,519 - INFO : scan_symptom_set_list : ['下肢无力', '困乏', '精神状态差', '双乳溢液', '呼吸困难']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-11-22 16:35:43,546 - INFO : test_bums : 5


**************************************************
current symptom: 0  下肢无力
Simi_Proba: 1.0000 , Simi_Text: 下肢无力
Simi_Proba: 0.9737 , Simi_Text: 四肢无力
Simi_Proba: 0.9692 , Simi_Text: 下肢的放射性无力感
Simi_Proba: 0.9370 , Simi_Text: 膝盖软弱无力
Simi_Proba: 0.9134 , Simi_Text: 走路时腿没有力气
Simi_Proba: 0.8988 , Simi_Text: 膀胱逼尿肌肌无力
Simi_Proba: 0.8924 , Simi_Text: 尿无力
Simi_Proba: 0.8809 , Simi_Text: 酸软无力
Simi_Proba: 0.8779 , Simi_Text: 按压时乏力
Simi_Proba: 0.8757 , Simi_Text: 手抖乏力
Simi_Proba: 0.8746 , Simi_Text: 腰部无力
Simi_Proba: 0.8737 , Simi_Text: 一侧胳膊上抬无力
Simi_Proba: 0.8670 , Simi_Text: 全身无力感
Simi_Proba: 0.8644 , Simi_Text: 乏力
Simi_Proba: 0.8633 , Simi_Text: 浑身松软无力
Simi_Proba: 0.8597 , Simi_Text: 双腿感觉没劲
Simi_Proba: 0.8584 , Simi_Text: 握拳乏力
Simi_Proba: 0.8554 , Simi_Text: 乏力纳差
Simi_Proba: 0.8470 , Simi_Text: 麻胀无力
Simi_Proba: 0.8435 , Simi_Text: 心脏无力
Simi_Proba: 0.8297 , Simi_Text: 气短乏力
Simi_Proba: 0.8166 , Simi_Text: 咽痛乏力
Simi_Proba: 0.8162 , Simi_Text: 咳嗽无力
Simi_Proba: 0.8133 , Simi_Text: 左手手指无力
Simi_Proba: 

In [19]:
##########################################
## Stage-2: Retrieve  Train-Corpus
##########################################

text_batch_list=["嗓子不舒服"]

test_res=simi_batch_match_info_infer(text_batch_list,standSymptomInfo,simi_model,
                                     thre=simi_threshold,top_k=top_k,batch_size=100)
print("test_res: ")
print(test_res)

for sub_key in test_res:
    sub_symptom=text_batch_list[sub_key]
    sub_res_list=test_res[sub_key]
    print("symptom: ",sub_symptom)
    for sub_res_tuple in sub_res_list:
        print("    matched symptom: ",standSymptomInfo.stand_text[sub_res_tuple[1]])
        print("    matched probability: ",sub_res_tuple[0])


test_res: 
{}


In [20]:
##########################################
## Stage-3: Retrieve  Template-Symptom-SET
##########################################

template_symptom_text="全身酸痛+咽喉痛+腹痛+咯黄痰+鼻痒+浑身酸疼+声音哑+嗓子哑+头痛+发烧+肚子疼+头疼+眼睛酸胀+干咳+鼻子痒+呼吸困难+睡不着+浑身疼痛+鼻塞+睡觉不好+肺结节+拉肚子+咳黄痰+嗓子有痰+易醒+黄痰+咽痛+甲流+咳嗽有痰+咽喉有异物感+多梦+嗓子有异物+发热+大腿酸痛+咯痰+感冒+打喷嚏+咳痰+流鼻涕+咽喉疼+嗓子疼+肚子痛+发冷+失眠+浑身发冷+浑身肌肉酸疼+呕吐+嗓子痒+咽干+咽喉有痰+头晕+流清鼻涕+嗓子痛+嗓子不舒服+喉咙痒+咳嗽+嗓子疼痛+流清涕+浑身酸痛+睡眠不好"
template_symptom_set_list=template_symptom_text.split("+")
print(len(template_symptom_set_list))
print(template_symptom_set_list[:5])

60
['全身酸痛', '咽喉痛', '腹痛', '咯黄痰', '鼻痒']


In [21]:
simi_threshold=0.80
top_k=120

scan_window_size=5
start_idx=55

scan_symptom_set_list=template_symptom_set_list[start_idx:start_idx+scan_window_size]
logger.info(f"scan_symptom_set_list : {scan_symptom_set_list}")
test_embed=simi_model.encode(scan_symptom_set_list)
faiss.normalize_L2(test_embed)
test_Proba, test_Id = symptom_set_faiss_index.search(test_embed, top_k)
logger.info(f"test_bums : {len(test_Proba)}")
sub_i=0
for sub_proba_list,sub_idx_list in zip(test_Proba,test_Id):
    print("*****"*10)
    print(f"current symptom: {sub_i}  {scan_symptom_set_list[sub_i]}")
    for sub_proba, sub_text_idx in zip(sub_proba_list,sub_idx_list):
        if sub_proba>=simi_threshold:
            print("Simi_Proba: {:.4f} , Simi_Text: {}".format(sub_proba,symptom_set_flatten_list[sub_text_idx]))
    sub_i+=1
    start_idx+=1

print("*****"*4)
print(f"Next start index: {start_idx}")
print("*****"*4)



2023-11-22 16:35:43,582 - INFO : scan_symptom_set_list : ['咳嗽', '嗓子疼痛', '流清涕', '浑身酸痛', '睡眠不好']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2023-11-22 16:35:43,612 - INFO : test_bums : 5


**************************************************
current symptom: 0  咳嗽
Simi_Proba: 1.0000 , Simi_Text: 咳嗽
Simi_Proba: 0.9897 , Simi_Text: 咳嗽严重
Simi_Proba: 0.9889 , Simi_Text: 反复咳嗽
Simi_Proba: 0.9877 , Simi_Text: 偶尔咳嗽
Simi_Proba: 0.9872 , Simi_Text: 咳嗽加重
Simi_Proba: 0.9855 , Simi_Text: 热咳嗽
Simi_Proba: 0.9838 , Simi_Text: 刺激性咳嗽
Simi_Proba: 0.9829 , Simi_Text: 咳嗽频率高
Simi_Proba: 0.9818 , Simi_Text: 咳嗽时间长
Simi_Proba: 0.9759 , Simi_Text: 咳嗽不止
Simi_Proba: 0.9722 , Simi_Text: 咳嗽持续时间较长
Simi_Proba: 0.9711 , Simi_Text: 天气变化时咳嗽严重
Simi_Proba: 0.9694 , Simi_Text: 咳嗽难受
Simi_Proba: 0.9690 , Simi_Text: 说话多时会咳嗽
Simi_Proba: 0.9631 , Simi_Text: 遇冷风时咳嗽加重
Simi_Proba: 0.9447 , Simi_Text: 咳嗽时有刺激感
Simi_Proba: 0.9437 , Simi_Text: 半夜剧咳
Simi_Proba: 0.9427 , Simi_Text: 呛到一点菜汁也会咳嗽
Simi_Proba: 0.9346 , Simi_Text: 清咳
Simi_Proba: 0.9310 , Simi_Text: 咳嗽频率每隔一两个小时
Simi_Proba: 0.9218 , Simi_Text: 咳嗦
Simi_Proba: 0.9194 , Simi_Text: 咳嗽三声
Simi_Proba: 0.9178 , Simi_Text: 咳得很剧烈
Simi_Proba: 0.9149 , Simi_Text: 激动时咳嗽
Simi_Pro